# SAKI Exercise 1

In [17]:
import pandas as pd
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.feature_extraction.text import CountVectorizer
import nltk
nltk.download("stopwords")
from nltk.corpus import stopwords

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\Achim\AppData\Roaming\nltk_data...
[nltk_data]   Unzipping corpora\stopwords.zip.


In [18]:
#Load transaction file from current working dir
df = pd.read_csv('Exercise1-TransactionClassification-DataSet.csv', error_bad_lines=False, delimiter=";").copy()

In [19]:
#check null values
df.isnull().sum()

Unnamed: 0                            0
Auftragskonto                        41
Buchungstag                           0
Valutadatum                           0
Buchungstext                          0
Verwendungszweck                      0
Beguenstigter/Zahlungspflichtiger     0
Kontonummer                           1
BLZ                                   1
Betrag                                0
Waehrung                              0
label                                 0
dtype: int64

In [20]:
#replace null values inside 'Kontonummer' and 'BLZ' with 0
df.Kontonummer.fillna(0, inplace = True)
df.BLZ.fillna(0, inplace = True)

In [21]:
df.isnull().sum()

Unnamed: 0                            0
Auftragskonto                        41
Buchungstag                           0
Valutadatum                           0
Buchungstext                          0
Verwendungszweck                      0
Beguenstigter/Zahlungspflichtiger     0
Kontonummer                           0
BLZ                                   0
Betrag                                0
Waehrung                              0
label                                 0
dtype: int64

In [22]:
#check if 'Betrag' is set correctly
df.Betrag.sum()

'2000.00-670.00-70.00-73.21-363200.00-19.99-28.85-300.00-50.002000.00-670.00200.00-25.99-19.99-73.21-10.00-300.00-28.85-70.00-250.002000.00-670.00200.00-25.99-19.99-73.21-300.002000.00-670.00-28.85-100.00200.00-300.002000.00-670.00-1000.00-10.00200.00-300.002000.00-670.00-28.85-50.00200.00-300.002000.00-4000.001792,73-5000-500-50-440,3-89,99-535-5-20-51,2-12,99-24,3-89,99-3,84-12,99-100-110-3260,04-158,9-100-412,161792,73-50-20-30-535-22,77-24,99-28,85-20-100-33,73-20-22,57-46-82,96-24-40-26,02100-67,8-18-1000-20-51,2-37-9,75-20-52,42-7,54-100-110-32-26,02-15,99-165,7-5,45-412,16-10,422986,24-50-26,3-37,19-7,99-6,73-10-12,97-19,95-24,99-32-140-35,99-10-25-46-52,5-113,4-40161,7-34-40-103,8-7,17-51,2-33-13,7-20,85-20-40,65-17,93-13,87-6,96-34,95-100-110-22,64-412,161792,73-17,94-30-89,95-39,95-24,9550-20-19,95-12,8-24,99-70-50-9,99-40-30-20275,98-20-5-19.99-28.85-50.002000.00-670.00200.00-25.99-19.99-73.21-10.00-300.00-28.85-70.00-250.002000.00-670.00200.00-25.99-19.99-73.21-300.002000.0

In [23]:
#convert 'Betrag' to float with '.' as spacer
df[df.Betrag.apply(lambda x: x.isnumeric())]
df.Betrag = [x.replace(',', '.') for x in df.Betrag]
df.Betrag = pd.to_numeric(df.Betrag,errors='ignore')
df.Betrag.sum()

825.8400000000001

In [30]:
count_vect = CountVectorizer()
df.Verwendungszweck = df.Verwendungszweck.str.replace('\d+', '')
feature_matrix_bow = count_vect.fit_transform(df.Verwendungszweck)
# Get feature names
feature_names = count_vect.get_feature_names()
# View feature names
feature_names
stop_words = set(stopwords.words('german'))
#stop_words
filtered_sentence = [w for w in feature_names if not w in stop_words] 
#filtered_sentence = [] 
#for w in word_tokens: 
#    if w not in stop_words: 
#        filtered_sentence.append(w) 
        
#print(word_tokens) 
print(filtered_sentence) 
#print(feature_names)
#print(clean_text)

['abonnement', 'abschlag', 'abschluss', 'adorsys', 'aenderungen', 'all', 'alld', 'amazon', 'ankenvers', 'apotheke', 'aramark', 'auslandsrei', 'auszahlung', 'bears', 'beitrags', 'bequ', 'beuthener', 'bil', 'bistro', 'blz', 'bp', 'bylademsbt', 'cafe', 'california', 'ccb', 'ccbadexxx', 'celona', 'classic', 'co', 'comix', 'company', 'danke', 'dauerauftrag', 'de', 'deco', 'deutschland', 'dietrich', 'dm', 'doris', 'edeka', 'einmalsparen', 'einzahlung', 'elv', 'em', 'end', 'erg', 'esso', 'eu', 'eur', 'euro', 'fil', 'folgenr', 'friends', 'fuerth', 'fuerthermare', 'ga', 'ganz', 'gas', 'gastro', 'gaststaette', 'geb', 'gehalt', 'geschenk', 'gmb', 'gmbh', 'gries', 'gruenstrom', 'gt', 'gutschrift', 'haushalt', 'horst', 'humanic', 'icherung', 'interne', 'intersport', 'karolinen', 'kd', 'kfn', 'kg', 'koestr', 'kundenreferenz', 'lastschrift', 'ld', 'lld', 'lombagine', 'mango', 'maxx', 'me', 'merewe', 'miete', 'mietzuschuss', 'mktplce', 'moda', 'neubauer', 'notprovided', 'nr', 'nsct', 'nuernb', 'nuernb

In [31]:
feature_matrix_bow.toarray()

array([[0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       ...,
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [1, 0, 0, ..., 0, 0, 0]], dtype=int64)

In [32]:
tfidfTransformer = TfidfTransformer()
feature_matrix_tfidf = tfidfTransformer.fit_transform(feature_matrix_bow)
feature_matrix_tfidf.toarray()

array([[0.        , 0.        , 0.        , ..., 0.        , 0.        ,
        0.        ],
       [0.        , 0.        , 0.        , ..., 0.        , 0.        ,
        0.        ],
       [0.        , 0.        , 0.        , ..., 0.        , 0.        ,
        0.        ],
       ...,
       [0.        , 0.        , 0.        , ..., 0.        , 0.        ,
        0.        ],
       [0.        , 0.        , 0.        , ..., 0.        , 0.        ,
        0.        ],
       [0.65319207, 0.        , 0.        , ..., 0.        , 0.        ,
        0.        ]])